# %pip install -qU langchain langchain-core langchain-community langgraph  langchain-openai beautifulsoup4 langchain-google-vertexai


In [2]:
%pip install -qU langchain langchain-core langchain-community langgraph  langchain-openai beautifulsoup4

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:

# 1) IMPORTS
import os
import getpass
import bs4
from typing_extensions import TypedDict, List

# LLM selection via LangChain's model-init helper
from langchain.chat_models import init_chat_model  # Doc: model init helper for chat models

# Vector store + docs
from langchain_core.vectorstores import InMemoryVectorStore  # Doc: in-memory vector store

from langchain_openai import OpenAIEmbeddings               # Doc: embeddings
from langchain_community.document_loaders import WebBaseLoader  # Doc: load webpage
from langchain_text_splitters import RecursiveCharacterTextSplitter  # Doc: chunking
from langchain_core.documents import Document  # Doc: structured document type


#Prompt + graph
from langchain import hub  # Doc: pull pre-built prompts
from langgraph.graph import START, StateGraph                 # Doc: LangGraph state graphs


In [7]:
# 2) KEYS (set your own; keep secrets safe!)
if not os.environ.get("OPENAI_API_KEY"):
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OPENAI_API_KEY: ")

# 3) CHOOSE MODELS (LLM + embeddings)
llm = init_chat_model("gpt-3.5-turbo", model_provider="openai")  # conversational LLM
embeddings = OpenAIEmbeddings(model="text-embedding-3-large")    # high-quality embeddings


In [12]:

# 4) LOAD & CHUNK CONTENT (we’ll index Lilian Weng’s agents blog from the LangChain tutorials)
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(parse_only=bs4.SoupStrainer(class_=("post-content", "post-title", "post-header"))),
    verify_ssl=False
)
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)


# 5) BUILD VECTOR STORE
vector_store = InMemoryVectorStore(embeddings)
_ = vector_store.add_documents(splits)

# 6) PROMPT (pre-built “RAG prompt” from LangChain Hub)
prompt = hub.pull("rlm/rag-prompt")



c:\Users\CAF9223\AppData\Local\Programs\Python\Python313\Lib\site-packages\urllib3\connectionpool.py:1097: InsecureRequestWarning: Unverified HTTPS request is being made to host 'lilianweng.github.io'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


APIConnectionError: Connection error.